In [ ]:
%env AIBS_RIG_ID=NP2

In [ ]:
import contextlib
import time
import requests
import yaml
import pathlib

import np_config
import np_jobs
import np_logging
import np_services
import np_session
import typing
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.openscope_barcode as Barcode

from np_services.resources.zro import ZroError 
import contextlib

logger = np_logging.getLogger()

np_workflows.elapsed_time_widget()

In [ ]:
# fetch required resources for running these things
def wrapped_get(url: str, **kwargs) -> requests.Response:
    """Wrap requests.get to raise for non-200 status codes."""
    response = requests.get(url, **kwargs)
    if response.status_code != 200:
        response.raise_for_status()
    return response


mtrain_regimen_response = wrapped_get(
    "https://raw.githubusercontent.com/AllenInstitute/mtrain_regimens/VisualBehaviorEPHYS_Task1G_v0.1.2/regimen.yml",
)
mtrain_regimen = yaml.safe_load(mtrain_regimen_response.text)
task_params = mtrain_regimen["stages"]["EPHYS_1_images_G_3uL_reward"]["parameters"]

commit_hash = '5adfa6e285774719135d0ebcba421f15f6f56168'
# script content to be write to files
behavior_script_content = requests.get(
    f'http://stash.corp.alleninstitute.org/projects/VB/repos/visual_behavior_scripts/raw/replay_session/behavior_script.py?at={commit_hash}'
).text.encode('utf8')

replay_script_content = requests.get(
    f'http://stash.corp.alleninstitute.org/projects/VB/repos/visual_behavior_scripts/raw/replay_session/replay_script.py?at={commit_hash}'
).text.encode('utf8')

local_behavior_script = pathlib.Path("./behavior_script.py")
local_behavior_script.write_text(behavior_script_content)
local_replay_script = pathlib.Path("./replay_script.py")
local_replay_script.write_text(replay_script_content)


class VBNMixin:

    """Multiple inheritance is a sin."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.imager = np_services.Imager()
        self._camstim_script_path = local_behavior_script
        self._is_behavior = True

    @property
    def camstim_script_path(self) -> str:
        return self._camstim_script_path
    @camstim_script_path.setter
    def camstim_script_path(self, path: str) -> None:
        self.c_amstim_script_path = path

    @property
    def is_behavior(self) -> bool:
        return self._is_behavior
    @is_behavior.setter
    def is_behavior(self, value: bool) -> None:
        self._is_behavior = value

    @property
    def camstim_params(self) -> dict[str, typing.Any]:
        return task_params

    def run_stim(self) -> None:
        self.update_state()
        
        if not np_services.ScriptCamstim.is_ready_to_start():
            raise RuntimeError("ScriptCamstim is not ready to start.")
        
        np_services.ScriptCamstim.script = self.camstim_script_path
        np_services.ScriptCamstim.params = self.camstim_params
        postfix = "behavior" if self.is_behavior else "replay"
        np_logging.web(f'sev_dev_{postfix}').info(f"Started camstim script")
        np_services.ScriptCamstim.start()
        
        with contextlib.suppress(Exception):
            while not np_services.ScriptCamstim.is_ready_to_start():
                time.sleep(2.5)
            
        if isinstance(np_services.ScriptCamstim, np_services.Finalizable):
            np_services.ScriptCamstim.finalize()

        with contextlib.suppress(Exception):
            np_logging.web(f'barcode_{postfix}').info(f"Finished session {self.mouse.mtrain.stage['name']}")


    def copy_data_files(self) -> None: 
        super().copy_data_files()
        # When all processing completes, camstim Agent class passes data and uuid to
        # /camstim/lims BehaviorSession class, and write_behavior_data() writes a
        # final .pkl with default name YYYYMMDDSSSS_mouseID_foragingID.pkl
        # - if we have a foraging ID, we can search for that
        if None == (stim_pkl := next(self.session.npexp_path.glob(f'{self.session.date:%y%m%d}*_{self.session.mouse}_*.pkl'), None)):
            logger.warning('Did not find stim file on npexp matching the format `YYYYMMDDSSSS_mouseID_foragingID.pkl`')
            return
        assert stim_pkl
        if not self.session.platform_json.foraging_id:
            self.session.platform_json.foraging_id = stim_pkl.stem.split('_')[-1]
        new_stem = f'{self.session.folder}.stim'
        logger.debug(f'Renaming stim file copied to npexp: {stim_pkl} -> {new_stem}')
        stim_pkl = stim_pkl.rename(stim_pkl.with_stem(new_stem))
        
        # remove other stim pkl, which is nearly identical, if it was also copied
        for pkl in self.session.npexp_path.glob('*.pkl'):
            if (
                self.session.folder not in pkl.stem
                and 
                abs(pkl.stat().st_size - stim_pkl.stat().st_size) < 1e6
            ):
                logger.debug(f'Deleting extra stim pkl copied to npexp: {pkl.stem}')
                pkl.unlink()


class Ephys(VBNMixin, np_workflows.PipelineEphys):
    def __init__(self, *args, **kwargs):
        self.services = (
            np_services.MouseDirector,
            np_services.Sync,
            np_services.VideoMVR,
            self.imager,
            np_services.NewScaleCoordinateRecorder,
            np_services.SessionCamstim,
            np_services.OpenEphys,
        )
        super().__init__(*args, **kwargs)

In [ ]:
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

## Select a trainer and mouse

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

#### Run this is mouse id has changed

In [ ]:
np_workflows.mtrain_widget(mouse)

## Generate new session

In [ ]:
experiment: np_workflows.PipelineExperiment = Ephys(mouse, user)

with contextlib.suppress(Exception):
    np_logging.web(f'barcode_{experiment.workflow.name.lower()}').info(f"{experiment} created")

session: np_session.PipelineSession = experiment.session
platform_json: np_session.PlatformJson = experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()

## Run behavior script

### Extend lick spout before behavior

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout()

In [ ]:
experiment.run_task()

## Run mapping script

### Retract lick spout before mapping

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()

In [ ]:
experiment.run_mapping()

## Run replay script

### Extend lick spout before replay

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout()

In [ ]:
# replace behavior script with replay script
experiment.camstim_script_path = local_replay_script
experiment.is_behavior = False
experiment.run_task()

## Run optotagging script

In [ ]:
experiment.run_optotagging()

## Finalize

In [ ]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_data_files()